# Building a simple autoencoder

Based on the ability to generate UK car registration numbers, one can build a dataset of training and test data. With a suitable vectorizer, one can go ahead and make an autoencoder.

### References

https://blog.keras.io/building-autoencoders-in-keras.html

In [1]:
import sys
import numpy as np
import keras
from keras import regularizers
from keras import layers
import random

if '..' not in sys.path:
    sys.path.append('..')
from car_reg_generator.car_reg_generator.uk_reg import UkRegGenerator
from car_reg_generator.car_reg_generator.uk_reg import UkRegBowVectorizer

### Data generation

In [2]:
n_train = 10000
n_test = 1000

random.seed(0)
g = UkRegGenerator()
v = UkRegBowVectorizer()

train_strs = [g.get_reg() for _ in range(n_train)]
train_vecs = np.array([v.vectorize(x) for x in train_strs])
test_strs = [g.get_reg() for _ in range(n_test)]
test_strs = [x for x in test_strs if x not in set(train_strs)]  # sanity check
test_vecs = np.array([v.vectorize(x) for x in test_strs])

vec_length = len(train_vecs[0])
print('input vector length = ' + str(vec_length))
print('actual number of test samples = ' + str(len(test_strs)))

input vector length = 252
actual number of test samples = 1000


### Standard autoencoder

Design the network. A single dense layer for the encoder and decoder needs an encoding dimension of around 100. Having more layers allows a smaller encoding dimension, although convergence during learning starts to become unstable.

In [3]:
encoding_dim = 30
input_reg = keras.Input(shape=(vec_length,))
# encoded = layers.Dense(encoding_dim, activation='relu', activity_regularizer=regularizers.l1(10e-5))(input_reg)
encoded = layers.Dense(vec_length, activation='relu')(input_reg)
encoded = layers.Dense(encoding_dim, activation='relu')(encoded)
encoded = layers.Dense(encoding_dim, activation='relu')(encoded)
encoder = keras.Model(input_reg, encoded)

encoded_input = keras.Input(shape=(encoding_dim,))
decoded = layers.Dense(encoding_dim, activation='relu')(encoded_input)
decoded = layers.Dense(vec_length, activation='relu')(decoded)
decoded = layers.Dense(vec_length, activation='sigmoid')(decoded)
decoder = keras.Model(encoded_input, decoded)

autoencoder = keras.Model(input_reg, decoder(encoder(input_reg)))

Do some learning. For shallow networks, almost any optimizer and loss work. For deeper networks, poor choices almost certainly yield failed convergence on the loss function. Even good choices yield convergence on the loss function only on some runs.

In [4]:
autoencoder.compile(optimizer='adam', loss='categorical_crossentropy')
autoencoder.fit(train_vecs, train_vecs,
                epochs=100,
                batch_size=256,
                shuffle=True,
                validation_data=(test_vecs, test_vecs))

Epoch 1/100
40/40 [==============================] - 1s 25ms/step - loss: 36.3656 - val_loss: 34.4921
Epoch 2/100
40/40 [==============================] - 1s 21ms/step - loss: 34.2818 - val_loss: 34.0305
Epoch 3/100
40/40 [==============================] - 1s 19ms/step - loss: 33.2773 - val_loss: 32.4146
Epoch 4/100
40/40 [==============================] - 1s 22ms/step - loss: 31.7591 - val_loss: 31.3422
Epoch 5/100
40/40 [==============================] - 1s 19ms/step - loss: 30.9186 - val_loss: 30.6825
Epoch 6/100
40/40 [==============================] - 1s 19ms/step - loss: 30.2745 - val_loss: 30.2219
Epoch 7/100
40/40 [==============================] - 1s 21ms/step - loss: 29.7318 - val_loss: 29.7818
Epoch 8/100
40/40 [==============================] - 1s 19ms/step - loss: 29.2281 - val_loss: 29.2920
Epoch 9/100
40/40 [==============================] - 1s 24ms/step - loss: 28.6255 - val_loss: 28.7015
Epoch 10/100
40/40 [==============================] - 1s 19ms/step - loss: 27.7816

40/40 [==============================] - 1s 22ms/step - loss: 13.7223 - val_loss: 13.7566
Epoch 81/100
40/40 [==============================] - 1s 21ms/step - loss: 13.7063 - val_loss: 13.7515
Epoch 82/100
40/40 [==============================] - 1s 21ms/step - loss: 13.6994 - val_loss: 13.7431
Epoch 83/100
40/40 [==============================] - 1s 26ms/step - loss: 13.6883 - val_loss: 13.7459
Epoch 84/100
40/40 [==============================] - 1s 23ms/step - loss: 13.6818 - val_loss: 13.7391
Epoch 85/100
40/40 [==============================] - 1s 24ms/step - loss: 13.6769 - val_loss: 13.7221
Epoch 86/100
40/40 [==============================] - 1s 19ms/step - loss: 13.6806 - val_loss: 13.7330
Epoch 87/100
40/40 [==============================] - 1s 25ms/step - loss: 13.6710 - val_loss: 13.7159
Epoch 88/100
40/40 [==============================] - 1s 20ms/step - loss: 13.6676 - val_loss: 13.7121
Epoch 89/100
40/40 [==============================] - 1s 27ms/step - loss: 13.6667 - v

Evaluation

In [5]:
decoded_regs = autoencoder.predict(test_vecs)
print(test_strs[:10])
print([v.recover(x) for x in decoded_regs[:10]])
acc = np.sum([v.recover(x) == y for x, y in zip(decoded_regs, test_strs)]) / len(test_strs)
print('accuracy = ' + str(acc))

['AN45FVI', 'LE23DCR', 'HA99JWO', 'EQ58XKI', 'RP49FNK', 'DJ61XIM', 'WO62NON', 'EA42JCE', 'DV47KTZ', 'HM70DFU']
['AN45FVI', 'LE23DCR', 'HA99JWO', 'EQ58XKI', 'RP49FNK', 'DJ61XIM', 'WO62NON', 'EA42JCE', 'DV47KTZ', 'HM70DFU']
accuracy = 1.0
